In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ../TileEM/

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM


In [14]:
cd analysis/TileEM/

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM


In [3]:
from TileEM_plot_toolbox import *
from qualityBaseline import *
from glob import glob

In [4]:
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
sampleN_lst=worker_Nbatches.keys()

In [16]:
from collections import OrderedDict
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)

In [94]:
cd ..

/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM


In [95]:
for Nworker in sampleN_lst:
    for batch_num in range(1,worker_Nbatches[Nworker]):
        dir_name = "{0}worker_rand{1}".format(Nworker,batch_num)
        #print dir_name
        os.chdir("sample/"+dir_name)
        # Summarization-Based Method 
        # Creating 2 empty precision recall table containing P,R for each metric used 
        cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']

        precision_df = pd.DataFrame(index=object_lst,columns=cols)
        recall_df = pd.DataFrame(index=object_lst,columns=cols)

        df.ix[df["Precision [Self]"]>1,"Precision [Self]"]=1
        df.ix[df["Recall [Self]"]>1,"Recall [Self]"]=1
        df = df[~(df["worker_id"].isin([1,2,3]))] #Filter our ground truth workers
        for attr in precision_df.keys():
            tbl=[]
            for objid in object_lst:
                workers=pkl.load(open("worker{}.pkl".format(objid)))
                filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
                best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
                tbl.append([objid,best_worker_BB["Precision [Self]"].values[0],best_worker_BB["Recall [Self]"].values[0]])
            PR_tbl = pd.DataFrame(tbl,columns=["objid","Precision","Recall"])
            precision_df[attr]=PR_tbl["Precision"]
            recall_df[attr]=PR_tbl["Recall"]
        precision_df.to_csv("precision_df.csv")
        recall_df.to_csv("recall_df.csv")
        os.chdir("../..")

5worker_rand1
5worker_rand2
5worker_rand3
5worker_rand4
5worker_rand5
5worker_rand6
5worker_rand7
5worker_rand8


IOError: [Errno 2] No such file or directory: 'worker8.pkl'

In [93]:
pwd

u'/Users/dorislee/Desktop/Research/seg/crowd-seg/analysis/TileEM/sample'

In [96]:
visionPR = pd.read_csv("../../Image-Segmentation/Vision-stuff/vision-baseline/PR.csv")

In [97]:
precision_df["Vision"] = visionPR["precision"]
recall_df["Vision"] = visionPR["recall"]

In [101]:
precision_df.mean()

Num Points          0.707535
Area Ratio          0.559025
Jaccard [Self]      0.954470
Precision [Self]    0.979881
Recall [Self]       0.729465
Vision              0.905204
dtype: float64

In [99]:
recall_df.mean()

Num Points          0.843395
Area Ratio          0.870589
Jaccard [Self]      0.950287
Precision [Self]    0.757573
Recall [Self]       0.989135
Vision              0.736919
dtype: float64

In [103]:
from tabulate import tabulate

In [134]:
print [list(precision_df.mean().as_matrix()),list(recall_df.mean().as_matrix())]

[[0.70753547589361709, 0.5590247110212766, 0.95446982951063797, 0.97988143444680853, 0.72946534988747957, 0.90520449921850243], [0.84339502145744694, 0.87058918721276601, 0.95028722610638316, 0.75757326877912845, 0.98913499846808495, 0.73691887132975531]]


In [143]:
tbl = pd.DataFrame([list(precision_df.mean().as_matrix()),list(recall_df.mean().as_matrix())],columns=precision_df.keys(),index=["Precision","Recall"])

In [146]:
tbl

,Num Points,Area Ratio,Jaccard [Self],Precision [Self],Recall [Self],Vision
Precision,0.707535,0.559025,0.954470,0.979881,0.729465,0.905204
Recall,0.843395,0.870589,0.950287,0.757573,0.989135,0.736919


In [144]:
# keys = list(precision_df.keys())
# keys.insert(0,"Metric")
# p_lst = list(precision_df.mean().as_matrix())
# p_lst.insert(0,"Precision")
# r_lst = list(recall_df.mean().as_matrix())
# r_lst.insert(0,"Recall")
# tbl=[keys,p_lst,r_lst]
# print tabulate(tbl,tablefmt='latex',floatfmt='.2f')

In [123]:
# tabulate(grouped_ji_fit[idx],showindex="False",headers='keys',tablefmt='latex',floatfmt='.2f')

In [147]:
print tabulate(tbl,tablefmt='latex',headers='keys',floatfmt='.2f')

\begin{tabular}{lrrrrrr}
\hline
           &   Num Points &   Area Ratio &   Jaccard [Self] &   Precision [Self] &   Recall [Self] &   Vision \\
\hline
 Precision &         0.71 &         0.56 &             0.95 &               0.98 &            0.73 &     0.91 \\
 Recall    &         0.84 &         0.87 &             0.95 &               0.76 &            0.99 &     0.74 \\
\hline
\end{tabular}


# Need to modify so that these are computed based on the selected sampled workers!